In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter

In [2]:
class Discriminator(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.disc = nn.Sequential(
            nn.Linear(in_features, 128), 
            nn.LeakyReLU(0.1),
            nn.Linear(128, 1), 
            nn.Sigmoid(),
        )
    
    def forward(self, x):
        return self.disc(x)

In [3]:
class Generator(nn.Module):
    def __init__(self, z_dim, img_dim):
        super().__init__()
        self.gen = nn.Sequential(
            nn.Linear(z_dim, 256), 
            nn.LeakyReLU(0.1), 
            nn.Linear(256, img_dim), 
            nn.Tanh(),
        )

    def forward(self, x):
        return self.gen(x)


In [4]:
# hyperparameter etc
device = "cuda" if torch.cuda.is_available() else "cpu"
lr = 3e-4
z_dim = 64 # 128, 256, etc
img_dim = 28*28*1 # 784
batch_size = 32
num_epochs = 50

disc = Discriminator(img_dim).to(device)
gen = Generator(z_dim, img_dim).to(device)
fixed_noise = torch.randn((batch_size, z_dim)).to(device)
transforms = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))]
)

In [5]:
dataset = datasets.MNIST(root="dataset/", transform=transforms, download=True)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
opt_disc = optim.Adam(disc.parameters(), lr=lr)
opt_gen = optim.Adam(gen.parameters(), lr=lr)
criterion = nn.BCELoss()
writer_fake = SummaryWriter(f"logs/GAN_MNIST/fake")
writer_real = SummaryWriter(f"logs/GAN_MNIST/real")
step = 0

In [6]:

for epoch in range(num_epochs):
    for batch_idx, (real, _) in enumerate(loader):
        real = real.view(-1, 784).to(device)
        batch_size = real.shape[0]

        # Train Discriminator: max log(D(real)) + log(1 - D(G(z)))
        noise = torch.randn(batch_size, z_dim).to(device)
        fake = gen(noise)
        disc_real = disc(real).view(-1)
        lossD_real = criterion(disc_real, torch.ones_like(disc_real))
        disc_fake = disc(fake).view(-1)
        lossD_fake = criterion(disc_fake, torch.zeros_like(disc_fake))
        lossD = (lossD_real + lossD_fake) / 2
        disc.zero_grad()
        lossD.backward(retain_graph=True)
        opt_disc.step()

        # Train Generator: min log(1 - D(G(z))) <-> max log(D(G(z))
        output = disc(fake).view(-1)
        lossG = criterion(output, torch.ones_like(output))
        gen.zero_grad()
        lossG.backward()
        opt_gen.step()


        if batch_idx == 0: 
            print(
                f"Epoch [ {epoch}/{num_epochs}] \ "
                f"Loss D: {lossD:.4f}, Loss G: {lossG:.4f}"
            )

            with torch.no_grad():
                fake = gen(fixed_noise).reshape(-1, 1, 28, 28)
                data = real.reshape(-1, 1, 28, 28)
                img_grid_fake = torchvision.utils.make_grid(fake, normalize = True)
                img_grid_real = torchvision.utils.make_grid(real, normalize= True)

                writer_fake.add_image(
                    "Mnist Fake Image", img_grid_fake, global_step = step
                )
                
                writer_real.add_image(
                    "Mnist real images", img_grid_real, global_step=step
                )

                step += 1

Epoch [ 0/50] \ Loss D: 0.7638, Loss G: 0.7177
Epoch [ 1/50] \ Loss D: 0.7639, Loss G: 0.7654
Epoch [ 2/50] \ Loss D: 0.3382, Loss G: 1.5519
Epoch [ 3/50] \ Loss D: 0.3690, Loss G: 1.2398
Epoch [ 4/50] \ Loss D: 1.0194, Loss G: 0.6052
Epoch [ 5/50] \ Loss D: 0.7015, Loss G: 0.8213
Epoch [ 6/50] \ Loss D: 0.5398, Loss G: 1.1518
Epoch [ 7/50] \ Loss D: 0.8920, Loss G: 0.6152
Epoch [ 8/50] \ Loss D: 0.5613, Loss G: 1.1807
Epoch [ 9/50] \ Loss D: 0.5502, Loss G: 1.2058
Epoch [ 10/50] \ Loss D: 0.8458, Loss G: 0.7921
Epoch [ 11/50] \ Loss D: 0.7137, Loss G: 0.8132
Epoch [ 12/50] \ Loss D: 0.7544, Loss G: 0.7912
Epoch [ 13/50] \ Loss D: 0.8955, Loss G: 0.7514
Epoch [ 14/50] \ Loss D: 0.6732, Loss G: 1.1716
Epoch [ 15/50] \ Loss D: 0.6487, Loss G: 1.0574
Epoch [ 16/50] \ Loss D: 0.6435, Loss G: 0.9813
Epoch [ 17/50] \ Loss D: 0.7133, Loss G: 0.9338
Epoch [ 18/50] \ Loss D: 0.8486, Loss G: 0.6331
Epoch [ 19/50] \ Loss D: 0.8652, Loss G: 0.7221
Epoch [ 20/50] \ Loss D: 0.6606, Loss G: 1.1544
Ep